In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import json
import validators
import datetime
import math
import re
import ipynb
import time
import warnings

In [3]:
warnings.filterwarnings("ignore")

In [4]:
def find_tables_flex(first, last, team, year):
    for i in range(0,8):
        try:
            if len(last) < 4:
                last = last + ('x' * (4 - len(last)))
            url = 'https://www.pro-football-reference.com/players/%s/%s%s0%i.htm' % (last[0], last[0:4], 
                                                                                     first[0:2], i)
            if first == 'Michael' and last == 'Thomas':
                url = 'https://www.pro-football-reference.com/players/T/ThomMi05.htm'
            if first == 'Antonio' and last == 'Brown':
                url = 'https://www.pro-football-reference.com/players/B/BrowAn04.htm'
            if first == 'A.J.' and last == 'Brown':
                url = 'https://www.pro-football-reference.com/players/B/BrowAJ00.htm'
            if first == 'Devin' and last == 'Hester':
                url = 'https://www.pro-football-reference.com/players/H/HestDe99.htm'
            response = requests.get(url)
            x = pd.read_html(url)
            df = x[0]
            df[df.columns[0]] = df[df.columns[0]].str.replace(r'[\*+]', '', regex=True)
            if (df[df[df.columns[0]] == str(year)][df.columns[2]]).values == team:
                return x[0], response, i + 1
        except:
            pass
    return 0, 0, i + 1

def find_age(response, year):
    soup = BeautifulSoup(response.content, "html.parser")
    x = soup.find('div', {'id': 'info'})
    dob = x.find('span"', {'id': 'necro-birth'})['data-birth']
    return math.floor(((datetime.date(year, 9, 7) - datetime.date(int(dob[0:4]), int(dob[5:7]), int(dob[8:10]))).days)/365)

In [5]:
wrs = pd.read_csv('wrs.csv')

In [6]:
wrs.drop(columns = 'Unnamed: 0', inplace = True)

In [7]:
wr2022 = pd.read_csv('2022wrs.csv')
corr_teams = pd.read_html('https://www.pro-football-reference.com/years/2022/fantasy.htm')
corrwr2022 = corr_teams[0][[('Unnamed: 1_level_0',  'Player'), ('Unnamed: 2_level_0',      'Tm'), ('Unnamed: 3_level_0', 'FantPos')]]

In [8]:
corrwr2022[('Unnamed: 1_level_0',  'Player')] = corrwr2022[('Unnamed: 1_level_0',  'Player')].str.replace(r'[\*+]', '', 
                                                                                                regex=True)

In [9]:
corrwr2022 = corrwr2022[corrwr2022[('Unnamed: 3_level_0', 'FantPos')] == 'WR']

In [10]:
wr2022.head(10)

,Rank,Player,REC,TGT,YDS,Y/R,LG,20+,TD,ATT,YDS.1,TD.1,FL,G,FPTS,FPTS/G,ROST
0,1.0,Justin Jefferson (MIN),128.0,184.0,"1,809",14.1,64.0,49.0,8.0,4.0,24.0,1.0,0.0,17.0,368.6,21.7,100.0%
1,2.0,Tyreek Hill (MIA),119.0,170.0,"1,710",14.4,64.0,45.0,7.0,7.0,32.0,1.0,0.0,17.0,341.2,20.1,100.0%
2,3.0,Davante Adams (LV),100.0,180.0,"1,516",15.2,60.0,47.0,14.0,3.0,-1.0,0.0,0.0,17.0,335.5,19.7,99.9%
3,4.0,Stefon Diggs (BUF),110.0,156.0,"1,455",13.2,53.0,35.0,11.0,1.0,-3.0,0.0,0.0,17.0,321.2,18.9,99.9%
4,5.0,CeeDee Lamb (DAL),107.0,156.0,"1,359",12.7,39.0,30.0,9.0,10.0,47.0,0.0,0.0,17.0,301.6,17.7,99.9%
5,6.0,A.J. Brown (PHI),88.0,145.0,"1,496",17.0,78.0,45.0,11.0,0.0,0.0,0.0,2.0,17.0,299.6,17.6,99.9%
6,7.0,Amon-Ra St. Brown (DET),106.0,146.0,"1,161",11.0,49.0,15.0,6.0,9.0,95.0,0.0,0.0,16.0,267.6,16.7,99.9%
7,8.0,Jaylen Waddle (MIA),75.0,117.0,"1,356",18.1,84.0,41.0,8.0,3.0,26.0,0.0,1.0,17.0,259.2,15.2,99.9%
8,9.0,DeVonta Smith (PHI),95.0,136.0,"1,196",12.6,45.0,30.0,7.0,0.0,0.0,0.0,1.0,17.0,254.6,15.0,99.2%
9,10.0,Amari Cooper (CLE),78.0,132.0,"1,160",14.9,55.0,31.0,9.0,0.0,0.0,0.0,0.0,17.0,247.0,14.5,97.0%


In [185]:
wr2022['Player'] = wr2022['Player'].replace('III', '', regex = True)
wr2022['Player'] = wr2022['Player'].replace('II', '', regex = True)
wr2022['Player'] = wr2022['Player'].replace('Jr.', '', regex = True)
wr2022['Player'] = wr2022['Player'].replace('Sr.', '', regex = True)
wr2022['Player'] = wr2022['Player'].replace(r'\bV\b', '', regex = True)
wr2022['Player'] = wr2022['Player'].replace(r'\bIV\b', '', regex = True)

In [186]:
x = wr2022['Player'].str.split(' ', expand = True)
# note the Amon and Equanimeous St. Brown exceptions
x[1] = x[1].replace('St.', 'St Brown', regex = True)

In [187]:
for i in range(0, len(x)):
    if x.loc[i,2] == "":
        x.loc[i,2] = x.loc[i,3]
x.drop(columns = 3, inplace = True)
x[2] = x[2].replace(r'[()]', '', regex = True)

In [188]:
wr2022[['First', 'Last', 'Tm']] = x
wr2022['Player'] = [i.strip() if isinstance(i,str) else i for i in wr2022['Player'].str.split('(', expand = True)[0]]
wr2022['Year'] = 2022
wr2022['Pos'] = 'WR'
wr2022.dropna(inplace = True)
wr2022['Age'] = None
wr2022.drop(columns = 'Rank', inplace = True)

In [189]:
x = pd.DataFrame()
x['Player'] = corrwr2022[('Unnamed: 1_level_0',  'Player')]
x['Tm'] = corrwr2022[('Unnamed: 2_level_0',      'Tm')]
x['Pos'] = corrwr2022[('Unnamed: 3_level_0', 'FantPos')]

In [190]:
wr2022 = pd.merge(wr2022, x, on = 'Player')

In [191]:
wr2022.drop(columns = ['Tm_x', 'Pos_x'], inplace = True)
wr2022.rename(columns = {'Tm_y': 'Tm', 'Pos_y': 'Pos'}, inplace = True)

In [192]:
column_order = ['First', 'Last', 'Year', 'Tm', 'Pos', 'Age', 'FPTS', 'Player']
wr2022 = wr2022.reindex(columns = column_order)
wr2022.rename(columns = {'FPTS': 'FantasyPoints'}, inplace = True)

In [193]:
totalwrs = pd.concat([wrs, wr2022])
totalwrs = totalwrs.sort_values('FantasyPoints', ascending = False).groupby('Year').head(100)
totalwrs = totalwrs.reset_index().drop(columns = 'index')

In [196]:
totalwrs

,First,Last,Year,Tm,Pos,Age,FantasyPoints,Player
0,Cooper,Kupp,2021,LAR,WR,28.0,440.0,Cooper Kupp
1,Randy,Moss,2007,NWE,WR,30.0,385.3,Randy Moss
2,Antonio,Brown,2014,PIT,WR,26.0,378.9,Antonio Brown
3,Antonio,Brown,2015,PIT,WR,27.0,378.2,Antonio Brown
4,Michael,Thomas,2019,NOR,WR,26.0,374.6,Michael Thomas
...,...,...,...,...,...,...,...,...
1795,Reggie,Brown,2008,PHI,WR,27.0,49.8,Reggie Brown
1796,Johnnie,Morton,2005,SFO,WR,34.0,49.8,Johnnie Morton
1797,Greg,Lewis,2008,PHI,WR,28.0,49.7,Greg Lewis
1798,Chaz,Schilens,2008,OAK,WR,23.0,49.4,Chaz Schilens


In [197]:
totalwrs.loc[(totalwrs['Last'] == 'Houshmandzadeh'), 'First'] = 'TJ'
totalwrs['First'] = totalwrs['First'].replace('D.K.', 'DK', regex = True)
totalwrs['First'] = totalwrs['First'].replace('K.J.', 'KJ', regex = True)
totalwrs['First'] = totalwrs['First'].replace("N'Keal", 'NKeal', regex = True)
totalwrs['Last'] = totalwrs['Last'].replace("St Brown", 'St.', regex = True)
totalwrs['Last'] = totalwrs['Last'].replace("\.", '', regex = True)

In [198]:
totalwrs['Targets'] = np.nan
totalwrs['Receptions'] = np.nan
totalwrs['RecYards'] = np.nan
totalwrs['RecTD'] = np.nan
totalwrs['Ctch%'] = np.nan
totalwrs['RushAtt'] = np.nan
totalwrs['RushYards'] = np.nan
totalwrs['RushTD'] = np.nan
totalwrs['Fumbles'] = np.nan
totalwrs['Experience'] = np.nan

In [199]:
totalwrs['Player'].value_counts()[totalwrs['Player'].value_counts() > 9]

Player
Larry Fitzgerald    16
Steve Smith         15
DeSean Jackson      12
Julio Jones         12
Emmanuel Sanders    12
Anquan Boldin       12
Brandon Marshall    11
Andre Johnson       11
A.J. Green          11
Roddy White         11
Randall Cobb        11
Nate Washington     10
Golden Tate         10
Marques Colston     10
T.Y. Hilton         10
Reggie Wayne        10
Robert Woods        10
Mike Williams       10
Vincent Jackson     10
Wes Welker          10
Danny Amendola      10
Jordy Nelson        10
DeAndre Hopkins     10
Antonio Brown       10
Demaryius Thomas    10
Name: count, dtype: int64

In [200]:
totalwrs[totalwrs['Player'] == 'Mike Williams']

,First,Last,Year,Tm,Pos,Age,FantasyPoints,Player,Targets,Receptions,RecYards,RecTD,Ctch%,RushAtt,RushYards,RushTD,Fumbles,Experience
194,Mike,Williams,2021,LAC,WR,27.0,247.00,Mike Williams,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
296,Mike,Williams,2010,TAM,WR,23.0,223.40,Mike Williams,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
328,Mike,Williams,2012,TAM,WR,25.0,217.72,Mike Williams,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
565,Mike,Williams,2018,LAC,WR,24.0,178.20,Mike Williams,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
575,Mike,Williams,2022,LAC,WR,NaN,176.50,Mike Williams,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
676,Mike,Williams,2019,LAC,WR,25.0,161.30,Mike Williams,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
720,Mike,Williams,2011,TAM,WR,24.0,156.40,Mike Williams,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
741,Mike,Williams,2020,LAC,WR,26.0,154.00,Mike Williams,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
754,Mike,Williams,2010,SEA,WR,26.0,152.10,Mike Williams,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1689,Mike,Williams,2005,DET,WR,21.0,66.00,Mike Williams,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
start = time.time()
for i in range(len(totalwrs)):
    if i % 300 == 0:
        print(i, totalwrs.loc[i, 'First'],totalwrs.loc[i, 'Last'])
    try:    
        stats, response, j = find_tables_flex(totalwrs.loc[i, 'First'],totalwrs.loc[i, 'Last'], 
                                       totalwrs.loc[i, 'Tm'], totalwrs.loc[i, 'Year'])
        stats[(( 'Unnamed: 0_level_0',  'Year'))] = pd.to_numeric(stats[(( 'Unnamed: 0_level_0',  'Year'))], errors = 'coerce')
        
        if totalwrs.loc[i, 'Tm'] == '2TM':
            idx = stats[(stats[( 'Unnamed: 2_level_0', 'Tm')] == '2TM') & (stats[( 'Unnamed: 0_level_0', 'Year')] == totalwrs.loc[i, 'Year'])].index + 1
            totalwrs.loc[i, 'Tm'] = stats.loc[idx, ( 'Unnamed: 2_level_0',    'Tm')].iloc[0]
        stats.dropna(subset = [(( 'Unnamed: 1_level_0',   'Age'))], inplace = True)
        
        totalwrs.loc[i, 'Experience'] = len(stats[stats[(( 'Unnamed: 0_level_0',  'Year'))] <= totalwrs.loc[i, 'Year']])
        
        if totalwrs.loc[i, 'Year'] == 2022:
            totalwrs.loc[i, 'Age'] = find_age(response, totalwrs.loc[i, 'Year'])

        date = stats[(stats[(( 'Unnamed: 0_level_0',  'Year'))] < (totalwrs.loc[i, 'Year'])) & (stats[(( 'Unnamed: 0_level_0',  'Year'))] >= (totalwrs.loc[i, 'Year'] - 2))].reset_index()
        date[('Receiving', 'Ctch%')] = date[('Receiving', 'Ctch%')].replace('%', '', regex = True)
        date.drop(columns = [( 'Unnamed: 2_level_0',     'Tm'), 
                             ( 'Unnamed: 3_level_0',     'Pos')], inplace = True)
        if ('Unnamed: 32_level_0', 'Awards') in date.columns:
            date.drop(columns = [('Unnamed: 32_level_0', 'Awards')], inplace = True)
        date = date.astype(float)
        
        totalwrs.loc[i, 'Targets'] = (date[('Receiving', 'Tgt')]*date[('Games', 'G')]*(date.index+1)).sum()/(date[('Games', 'G')]*(date.index+1)).sum()
        totalwrs.loc[i, 'Receptions'] = (date[('Receiving', 'Rec')]*date[('Games', 'G')]*(date.index+1)).sum()/(date[('Games', 'G')]*(date.index+1)).sum()
        totalwrs.loc[i, 'RecYards'] = (date[('Receiving', 'Yds')]*date[('Games', 'G')]*(date.index+1)).sum()/(date[('Games', 'G')]*(date.index+1)).sum()
        totalwrs.loc[i, 'RecTD'] = (date[('Receiving', 'TD')]*date[('Games', 'G')]*(date.index+1)).sum()/(date[('Games', 'G')]*(date.index+1)).sum()
        totalwrs.loc[i, 'Ctch%'] = (date[('Receiving', 'Ctch%')]*date[('Games', 'G')]*(date.index+1)).sum()/(date[('Games', 'G')]*(date.index+1)).sum()
        totalwrs.loc[i, 'RushAtt'] = (date[('Rushing', 'Att')]*date[('Games', 'G')]*(date.index+1)).sum()/(date[('Games', 'G')]*(date.index+1)).sum()
        totalwrs.loc[i, 'RushYards'] = (date[('Rushing', 'Yds')]*date[('Games', 'G')]*(date.index+1)).sum()/(date[('Games', 'G')]*(date.index+1)).sum()
        totalwrs.loc[i, 'RushTD'] = (date[('Rushing', 'TD')]*date[('Games', 'G')]*(date.index+1)).sum()/(date[('Games','G')]*(date.index+1)).sum()
        totalwrs.loc[i, 'Fumbles'] = (date[('Unnamed: 30_level_0', 'Fmb')]*date[('Games', 'G')]*(date.index+1)).sum()/(date[('Games', 'G')]*(date.index+1)).sum()
    except:
        print("Error:", totalwrs.loc[i, 'First'], totalwrs.loc[i, 'Last'], totalwrs.loc[i, 'Tm'], totalwrs.loc[i, 'Year'])
    time.sleep(7.5 * j)
print(time.time() - start)

0 Cooper Kupp
300 Chris Godwin
Error: Terrelle Pryor CLE 2016
Error: Mike Sims-Walker JAX 2009
600 Julian Edelman
Error: Mike Sims-Walker JAX 2010
900 Courtland Sutton
Error: David Moore SEA 2020


In [173]:
totalwrs.dropna(inplace = True)

In [174]:
totalwrs.to_csv('finalwrs.csv')

In [1]:
totalwrs

NameError: name 'totalwrs' is not defined